# Composition

### Lifted mostly from Learn Python the Hard Way

http://learnpythonthehardway.org/book/ex44.html

In [5]:
class Other(object):
    
    def __init__(self):
        print("Other __init__")
        
    def override(self):
        print("Other override()")
        
    def implicit(self):
        print("Other implicit()")
        
    def altered(self):
        print("Other altered()")

In [59]:
class MyComposedClass(object):
    """ I use Other, but I do not inherit from it """
    
    def __init__(self):
        self.other = Other()
        
    def implicit(self):
        '''
        I do nothing myself, I delegate
        '''
        self.other.implicit()
        
    def override(self):
        """
        I simply choose to ignore self.other.override()
        and instead do the work myself
        """
        print("MyComposedClass override()")
        
    def altered(self):
        """
        I do some work myself, yet also delegate
        """
        print("MyComposedClass, BEFORE OTHER altered()")
        self.other.altered()
        print("MyComposedClass, AFTER OTHER altered()")

In [60]:
my_composed_class = MyComposedClass()

Other __init__


In [61]:
my_composed_class.implicit()

Other implicit()


In [62]:
my_composed_class.override()

MyComposedClass override()


In [63]:
my_composed_class.altered()

MyComposedClass, BEFORE OTHER altered()
Other altered()
MyComposedClass, AFTER OTHER altered()


# Properties

In [12]:
my_class = MyClass()

In [13]:
my_class.get_x()

5

In [14]:
my_class.set_x(4)

In [15]:
my_class.set_x(11)

ValueError: Whoa, what are you thinking?

### Okay, basic stuff here... what's the big deal?  What if you have a few instances of MyClass floating around and you want to do some work with their attributes....

In [16]:
my_other_class = MyClass()

In [17]:
my_class.set_x(my_class.get_x() + my_other_class.get_x())

ValueError: Whoa, what are you thinking?

In [18]:
my_class.get_x()

4

### That worked, but it sure was ugly.  Note that the throwing of the exception was deliberate and planned! I set x to a value outside its perscribed range.  The uglyness is in having to access an attribute via a method call.  I'd rather do something like this...

In [19]:
my_class._x = my_class._x + my_other_class._x

### That was easy, the syntax clear, but I now have a value for x in my_class that is out of range: x should never be equal to or greater than 9! Oh no! The internal consistancy of my object has been violated! Mayhem ensues....

In [64]:
my_class._x

9

## Enter properties!

In [22]:
help(property)

Help on class property in module builtins:

class property(object)
 |  property(fget=None, fset=None, fdel=None, doc=None) -> property attribute
 |  
 |  fget is a function to be used for getting an attribute value, and likewise
 |  fset is a function for setting, and fdel a function for del'ing, an
 |  attribute.  Typical use is to define a managed attribute x:
 |  
 |  class C(object):
 |      def getx(self): return self._x
 |      def setx(self, value): self._x = value
 |      def delx(self): del self._x
 |      x = property(getx, setx, delx, "I'm the 'x' property.")
 |  
 |  Decorators make defining new properties or modifying existing ones easy:
 |  
 |  class C(object):
 |      @property
 |      def x(self):
 |          "I am the 'x' property."
 |          return self._x
 |      @x.setter
 |      def x(self, value):
 |          self._x = value
 |      @x.deleter
 |      def x(self):
 |          del self._x
 |  
 |  Methods defined here:
 |  
 |  __delete__(self, instance, /)
 |  

In [30]:
class MyNewClass(object):
    
    def __init__(self):
        self._x = 5
        self._y = 33
        
    def get_x(self):
        return self._x
    
    def set_x(self, x):
        if 2 < x < 9:
            self._x = x
        else:
            raise ValueError("Whoa, what are you thinking?")
            
    x = property(get_x, set_x)

In [26]:
my_new_class = MyNewClass()

In [27]:
my_new_class.x

5

In [28]:
my_new_class.x = 4

In [29]:
my_new_class.x

4

In [31]:
my_new_class.x = 99

ValueError: Whoa, what are you thinking?

In [32]:
my_new_class.x = my_new_class.x + my_other_class.x

AttributeError: 'MyClass' object has no attribute 'x'

### Let's refactor the class to use decorators rather than a call to the property() built-in

In [47]:
class MyDecoratedClass(object):
    
    def __init__(self):
        self._x = 5
        self._y = 33
    
#     def get_x(self):
#         return self._x

    @property
    def x(self):
        return self._x
    
#     def set_x(self, x):
#         if 2 < x < 9:
#             self._x = x
#         else:
#             raise ValueError("Whoa, what are you thinking?")

    @x.setter
    def x(self, x):
        if 2 < x < 9:
            self._x = x
        else:
            raise ValueError("Whoa, what are you thinking?")

    def make_me_a_sandwhich(self):
        pass

    # x = property(get_x, set_x)

In [48]:
my_decorated_class = MyDecoratedClass()

In [49]:
my_decorated_class.x = my_decorated_class.x + 1

In [50]:
my_decorated_class.x

6

In [51]:
my_decorated_class.x += 1

In [52]:
my_decorated_class.x

7

### To create a read-only attribute... don't define the setter

In [53]:
class MyReadOnlyClass(object):
    
    def __init__(self):
        self._x = 5
        
    @property
    def x(self):
        return self._x

In [54]:
my_read_only = MyReadOnlyClass()

In [55]:
my_read_only.x

5

In [56]:
my_read_only.x = 6

AttributeError: can't set attribute